In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 24 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

04 April 2023 ; 01:26:18


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,4.00,116.18,112.32,3.85,3.90,116.08,112.30,3.77,1,1
12,MarJon Beauchamp,SF,22,MIL,48,9,12.8,1.8,4.5,0.401,...,4.00,116.18,112.32,3.85,3.90,116.08,112.30,3.77,1,1
18,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.33,117.73,117.44,0.29,0.16,117.70,117.59,0.11,1,1
26,Jaden Hardy,SG,20,DAL,45,3,14.4,2.9,6.6,0.438,...,0.33,117.73,117.44,0.29,0.16,117.70,117.59,0.11,1,1
39,AJ Griffin,SF,19,ATL,69,11,19.2,3.4,7.3,0.466,...,-0.01,117.30,117.32,-0.02,-0.09,117.46,117.55,-0.09,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,15.0,0.346479,2.771831,6.6,20.0,11,0.262783
1,Walker Kessler,23.0,UTA,21.6,2.327027,7.867568,4.0,9.2,12,0.192914
2,Jaden Ivey,30.7,DET,11.9,-0.820000,-0.117143,5.3,15.8,15,0.094772
3,Bennedict Mathurin,28.2,IND,13.3,-0.874667,1.968000,5.3,16.6,13,0.074411
4,Jalen Duren,24.5,DET,17.4,0.650794,5.466667,3.7,8.9,15,0.072276


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)